In [18]:
# %%bash
# conda create -n mol_transformer python=3.5
# source activate mol_transformer
# conda install rdkit -c rdkit
# conda install future six tqdm pandas
# conda install pytorch=0.4.1 torchvision -c pytorch
# pip install torchtext==0.3.1
# pip install -e .
# conda install -c anaconda ipykernel
# python -m ipykernel install --user --name=mol_transformer

Error while terminating subprocess (pid=6668): 


In [31]:
# Tokenization (already done in /mol_transf/MIT_mixed)
# def smi_tokenizer(smi):
#     """
#     Tokenize a SMILES molecule or reaction
#     """
#     import re
#     pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
#     regex = re.compile(pattern)
#     tokens = [token for token in regex.findall(smi)]
#     assert smi == ''.join(tokens)
#     return ' '.join(tokens)

CCS(=O)(=O)Cl.OCCBr>CCN(CC)CC.CCOCC>CCS(=O)(=O)OCCBr
C C S ( = O ) ( = O ) Cl . O C C Br > C C N ( C C ) C C . C C O C C > C C S ( = O ) ( = O ) O C C Br

CC(C)CS(=O)(=O)Cl.OCCCl>CCN(CC)CC.CCOCC>CC(C)CS(=O)(=O)OCCCl
C C ( C ) C S ( = O ) ( = O ) Cl . O C C Cl > C C N ( C C ) C C . C C O C C > C C ( C ) C S ( = O ) ( = O ) O C C Cl

O=[N+]([O-])c1cccc2cnc(Cl)cc12>CC(=O)O.O.[Fe].[Na+].[OH-]>Nc1cccc2cnc(Cl)cc12
O = [N+] ( [O-] ) c 1 c c c c 2 c n c ( Cl ) c c 1 2 > C C ( = O ) O . O . [Fe] . [Na+] . [OH-] > N c 1 c c c c 2 c n c ( Cl ) c c 1 2

Cc1cc2c([N+](=O)[O-])cccc2c[n+]1[O-].O=P(Cl)(Cl)Cl>>Cc1cc2c([N+](=O)[O-])cccc2c(Cl)n1
C c 1 c c 2 c ( [N+] ( = O ) [O-] ) c c c c 2 c [n+] 1 [O-] . O = P ( Cl ) ( Cl ) Cl > > C c 1 c c 2 c ( [N+] ( = O ) [O-] ) c c c c 2 c ( Cl ) n 1

CCCCC[C@H](O)C=CC1C=CC(=O)C1CC=CCCCC(=O)O>CCO.Cl>CCCCC[C@H](O)C=CC1CCC(=O)C1CC=CCCCC(=O)O
C C C C C [C@H] ( O ) C = C C 1 C = C C ( = O ) C 1 C C = C C C C C ( = O ) O > C C O . Cl > C C C C C [C@H] ( O ) C = C C 1 C 

In [ ]:
# Augmentation (already done in /mol_transf/MIT_mixed_augm)
# from rdkit import Chem
# smi = ''
# random_equivalent_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smi), doRandom=True)

In [8]:
%%bash
# Input file generation (from augmented data)
dataset=MIT_mixed_augm # MIT_mixed_augm / STEREO_mixed_augm

python preprocess.py -train_src data/${dataset}/src-train.txt \
                     -train_tgt data/${dataset}/tgt-train.txt \
                     -valid_src data/${dataset}/src-val.txt \
                     -valid_tgt data/${dataset}/tgt-val.txt \
                     -save_data data/eric_mol_transf/${dataset}/${dataset} \
                     -src_seq_length 1000 -tgt_seq_length 1000 \
                     -src_vocab_size 1000 -tgt_vocab_size 1000 -share_vocab

# We use a shared vocabulary. The vocab_size and seq_length are chosen to include the whole datasets.

Traceback (most recent call last):
  File "preprocess.py", line 13, in <module>
    import torch
ImportError: No module named torch


CalledProcessError: Command 'b'# Input file generation (from augmented data)\ndataset=MIT_mixed_augm # MIT_mixed_augm / STEREO_mixed_augm\n\npython preprocess.py -train_src ../data/eric_mol_transf/${dataset}/src-train.txt \\\n                     -train_tgt ../data/eric_mol_transf/${dataset}/tgt-train.txt \\\n                     -valid_src ../data/eric_mol_transf/${dataset}/src-val.txt \\\n                     -valid_tgt ../data/eric_mol_transf/${dataset}/tgt-val.txt \\\n                     -save_data ../data/eric_mol_transf/${dataset}/${dataset} \\\n                     -src_seq_length 1000 -tgt_seq_length 1000 \\\n                     -src_vocab_size 1000 -tgt_vocab_size 1000 -share_vocab\n\n# We use a shared vocabulary. The vocab_size and seq_length are chosen to include the whole datasets.\n'' returned non-zero exit status 1.

In [ ]:
# Training phase (from preprocessed augmented data)
dataset=MIT_mixed_augm # MIT_mixed_augm / STEREO_mixed_augm

# Our MIT models were trained for 48 hours on a single GPU (STEREO for 72h), using the following hyperparameters:
python  train.py -data data/${dataset}/${dataset} \
                   -save_model experiments/checkpoints/${dataset}/${dataset}_model \
                   -seed 42 -gpu_ranks 0 -save_checkpoint_steps 10000 -keep_checkpoint 20 \
                   -train_steps 500000 -param_init 0  -param_init_glorot -max_generator_batches 32 \
                   -batch_size 4096 -batch_type tokens -normalization tokens -max_grad_norm 0  -accum_count 4 \
                   -optim adam -adam_beta1 0.9 -adam_beta2 0.998 -decay_method noam -warmup_steps 8000  \
                   -learning_rate 2 -label_smoothing 0.0 -report_every 1000 \
                   -layers 4 -rnn_size 256 -word_vec_size 256 -encoder_type transformer -decoder_type transformer \
                   -dropout 0.1 -position_encoding -share_embeddings \
                   -global_attention general -global_attention_function softmax -self_attn_type scaled-dot \
                   -heads 8 -transformer_ff 2048

# To achieve the best results with single models, we average the last 20 checkpoints.

In [ ]:
# Testing phase (from averaged model and MIT mixed augmented data)
model=${dataset}_model_average_20.pt

python translate.py -model experiments/models/${model} \
                    -src data/${dataset}/eric_test/short_src-test.txt \
                    -output experiments/results/eric_results/predictions_${model}_on_${dataset}_test.txt \
                    -batch_size 64 -replace_unk -max_length 200 -fast

In [ ]:
# Evaluation phase (from averaged model and MIT mixed augmented data)
# Run the following script to get the top-1 accuracy.

python score_predictions.py -targets data/${dataset}/eric_test/short_tgt-test.txt \
                    -predictions experiments/results/eric_results/predictions_${model}_on_${dataset}_test.txt -beam_size 1